# Perch Routing

Vehicle Routing problem for volunteer deliveries for Perch's lockdown dinner dash.

### Constraints
For each day:
* Fixed number of delivery locations at certain addresses
* Fixed starting point to collect deliveries from
* Each volunteer ends at their own home address
* Each volunteer has a class of capacity depending on their vehicle type (bicycle, motorcycle, car)
* Each delivery can be for a number of meals for a number of people (i.e. variable weight per delivery per address)
* Number of meals per person depends on day of the week (each person given 2 meals on MonWed, 3 on Fri)

Also:
* Optimum route could be traffic dependant for cars
* Availability of volunteers different each day (i.e not always the same volunteers each day)
* Delivery addresses change each day (not always teh same people receiving deliveries every day)



In [2]:
import sys
sys.path

['/Users/louisekirkham/Documents/personal/perch_vrp',
 '/Users/louisekirkham/.pyenv/versions/3.7.3/lib/python37.zip',
 '/Users/louisekirkham/.pyenv/versions/3.7.3/lib/python3.7',
 '/Users/louisekirkham/.pyenv/versions/3.7.3/lib/python3.7/lib-dynload',
 '',
 '/Users/louisekirkham/.local/lib/python3.7/site-packages',
 '/Users/louisekirkham/.pyenv/versions/3.7.3/lib/python3.7/site-packages',
 '/Users/louisekirkham/.pyenv/versions/3.7.3/lib/python3.7/site-packages/IPython/extensions',
 '/Users/louisekirkham/.ipython']

In [42]:
# Install a package direcyly from this notebook
import sys
!{sys.executable} -m pip install nbformat

In [41]:
from __future__ import division
from __future__ import print_function
import os
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import googlemaps
from itertools import tee
import requests
import json
import urllib.request
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

In [12]:
os.getcwd()

'/Users/louisekirkham/Documents/personal/perch_vrp'

## Input Data

In [15]:
# Delivery addresses
delivery_points_df = pd.read_csv('data/input/10_random_delivery_points.csv', index_col=False)
delivery_points_df

,id_num,name,address,postcode,latitude,longitude
0,delivery_01,Gia Marrgaret,"94 Woolridge Way, Hackney, London",E9 6PR,51.542348,-0.049885
1,delivery_02,Sylvan Esso,"7 Poole Rd, London",E9 7AE,51.543422,-0.045961
2,delivery_03,Eta James,"51 Whiston Road, London",E2 8RZ,51.534488,-0.066195
3,delivery_04,Chet Baker,"Adeyfield House, Cranwood St, Hoxton, London",EC1V 9PD,51.526541,-0.086816
4,delivery_05,Louis Armstrong,"131 Highbury Quadrant Highbury East, London",N5 2TG,51.558769,-0.095979
5,delivery_06,Aretha Franklin,"23 Fladbury Rd, London",N15 6SB,51.578274,-0.085052
6,delivery_07,Akira Kosemura,"10 Ladbroke House 62-66 Highbury Grove, Highbu...",N5 2AG,51.553173,-0.097935
7,delivery_08,Ella Fitzgerald,"45 Lampard Grove, Cazenove, London",N16 6XB,51.567425,-0.070328
8,delivery_09,Nancy Wilson,"17 Harrowgate Rd, London",E9 7BS,51.543033,-0.038413
9,delivery_10,Alice Coltrane,"22 Turner House, Corbyn St, Finsbury Park, London",N4 3DD,51.568494,-0.117942


In [16]:
# Collection point
class CollectionPoint:
    '''This is a class for collection points'''
    def __init__(self, id_num, name, address, postcode, latitude, longitude):
        self.id_num = id_num
        self.name = name
        self.address = address
        self.postcode = postcode
        self.latitude = latitude
        self.longitude = longitude
        
    def to_dict(self):
        return {
            'id_num': self.id_num,
            'name': self.name,
            'address': self.address,
            'postcode': self.postcode,
            'latitude': self.latitude,
            'longitude': self.longitude
        }

canababes = CollectionPoint('collect_01',
                            'Canababes',
                            'Unit 8 Hamlet Industrial Estate, 96 White Post Ln, London',
                            'E9 5EN',
                            '51.542540',
                            '-0.023000')

In [17]:
collection_point_df = pd.DataFrame.from_records([s.to_dict() for s in [canababes]])
collection_point_df

,id_num,name,address,postcode,latitude,longitude
0,collect_01,Canababes,"Unit 8 Hamlet Industrial Estate, 96 White Post...",E9 5EN,51.542540,-0.023000


In [18]:
# Dispatch crew assemble!
dispatch_crew_df = pd.read_csv('data/input/dispatch_crew.csv', index_col=False)
dispatch_crew_df


,id_num,name,address,postcode,latitude,longitude
0,dispatch_01,Louise Kirkham,25 Coniston Walk London,E9 6EP,51.550955,-0.047934
1,dispatch_02,Erykah Badu,"32 Lansdowne Dr, Hackney, London",E8 3EG,51.543314,-0.065094


In [22]:
my_mapbox_access_token = 'pk.eyJ1IjoibG91aXNla2lya2hhbSIsImEiOiJjazYxMGJrZDcwOTdzM3RxaXQ4NG1jZHVlIn0.NeE7I8fu5zglEoeFc_eirQ'

fig = go.Figure()

# Collection Point
fig.add_trace(go.Scattermapbox(
        lat=[canababes.latitude],
        lon=[canababes.longitude],
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=13,
            color='#D05146',
        ),
        hoverinfo='text',
        text=canababes.name,
        name='Collection Point'
    ))

# Delivery Points
fig.add_trace(go.Scattermapbox(
        lat=delivery_points_df['latitude'],
        lon=delivery_points_df['longitude'],
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=13,
            color='#12727D',
        ),
        hoverinfo='text',
        text=delivery_points_df['address'],
        name='Delivery Point'
    ))

# Dispatchers start/end points
fig.add_trace(go.Scattermapbox(
        lat=dispatch_crew_df['latitude'],
        lon=dispatch_crew_df['longitude'],
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=13,
            color='#C8A3B5',
        ),
        hoverinfo='text',
        text=dispatch_crew_df['address'],
        name='Dispatcher Start/End Point',
    ))

# Map layout
fig.update_layout(
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        accesstoken=my_mapbox_access_token,
        style='light',
        center={'lon': np.mean(delivery_points_df['longitude']),
                'lat': np.mean(delivery_points_df['latitude'])},
        zoom=11
    ),
)

fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

## Distance Matrix API

In [23]:
# Set up API key for google distance matrix API
API_KEY='xyz'

In [24]:
# Need to combine all locations (collection point, dispatcher points and delivery points) into a single dataframe

# Combine delivery and dispatch dataframes
all_locations_df = pd.concat([collection_point_df, delivery_points_df, dispatch_crew_df, ]).reset_index(drop=True)
all_locations_df

,id_num,name,address,postcode,latitude,longitude
0,collect_01,Canababes,"Unit 8 Hamlet Industrial Estate, 96 White Post...",E9 5EN,51.542540,-0.023000
1,delivery_01,Gia Marrgaret,"94 Woolridge Way, Hackney, London",E9 6PR,51.542348,-0.049885
2,delivery_02,Sylvan Esso,"7 Poole Rd, London",E9 7AE,51.543422,-0.045961
3,delivery_03,Eta James,"51 Whiston Road, London",E2 8RZ,51.534488,-0.066195
4,delivery_04,Chet Baker,"Adeyfield House, Cranwood St, Hoxton, London",EC1V 9PD,51.526541,-0.086816
5,delivery_05,Louis Armstrong,"131 Highbury Quadrant Highbury East, London",N5 2TG,51.558769,-0.095979
6,delivery_06,Aretha Franklin,"23 Fladbury Rd, London",N15 6SB,51.578274,-0.085052
7,delivery_07,Akira Kosemura,"10 Ladbroke House 62-66 Highbury Grove, Highbu...",N5 2AG,51.553173,-0.097935
8,delivery_08,Ella Fitzgerald,"45 Lampard Grove, Cazenove, London",N16 6XB,51.567425,-0.070328
9,delivery_09,Nancy Wilson,"17 Harrowgate Rd, London",E9 7BS,51.543033,-0.038413


In [25]:
# Add address for distance matrix api column

all_locations_df['api_address'] = all_locations_df['address'] + ' ' + all_locations_df['postcode']
all_locations_df['api_address'] = all_locations_df['api_address'].str.replace(',', '').str.replace(' ', '+')
all_locations_df

,id_num,name,address,postcode,latitude,longitude,api_address
0,collect_01,Canababes,"Unit 8 Hamlet Industrial Estate, 96 White Post...",E9 5EN,51.542540,-0.023000,Unit+8+Hamlet+Industrial+Estate+96+White+Post+...
1,delivery_01,Gia Marrgaret,"94 Woolridge Way, Hackney, London",E9 6PR,51.542348,-0.049885,94+Woolridge+Way+Hackney+London+E9+6PR
2,delivery_02,Sylvan Esso,"7 Poole Rd, London",E9 7AE,51.543422,-0.045961,7+Poole+Rd+London+E9+7AE
3,delivery_03,Eta James,"51 Whiston Road, London",E2 8RZ,51.534488,-0.066195,51+Whiston+Road+London+E2+8RZ
4,delivery_04,Chet Baker,"Adeyfield House, Cranwood St, Hoxton, London",EC1V 9PD,51.526541,-0.086816,Adeyfield+House+Cranwood+St+Hoxton+London+EC1V...
5,delivery_05,Louis Armstrong,"131 Highbury Quadrant Highbury East, London",N5 2TG,51.558769,-0.095979,131+Highbury+Quadrant+Highbury+East+London+N5+2TG
6,delivery_06,Aretha Franklin,"23 Fladbury Rd, London",N15 6SB,51.578274,-0.085052,23+Fladbury+Rd+London+N15+6SB
7,delivery_07,Akira Kosemura,"10 Ladbroke House 62-66 Highbury Grove, Highbu...",N5 2AG,51.553173,-0.097935,10+Ladbroke+House+62-66+Highbury+Grove+Highbur...
8,delivery_08,Ella Fitzgerald,"45 Lampard Grove, Cazenove, London",N16 6XB,51.567425,-0.070328,45+Lampard+Grove+Cazenove+London+N16+6XB
9,delivery_09,Nancy Wilson,"17 Harrowgate Rd, London",E9 7BS,51.543033,-0.038413,17+Harrowgate+Rd+London+E9+7BS


In [26]:
# Simplify problem, ignore dispatcher addresses for now:

basic_locations_df = all_locations_df[~all_locations_df['id_num'].str.contains('dispatch')]
basic_locations_df

,id_num,name,address,postcode,latitude,longitude,api_address
0,collect_01,Canababes,"Unit 8 Hamlet Industrial Estate, 96 White Post...",E9 5EN,51.542540,-0.023000,Unit+8+Hamlet+Industrial+Estate+96+White+Post+...
1,delivery_01,Gia Marrgaret,"94 Woolridge Way, Hackney, London",E9 6PR,51.542348,-0.049885,94+Woolridge+Way+Hackney+London+E9+6PR
2,delivery_02,Sylvan Esso,"7 Poole Rd, London",E9 7AE,51.543422,-0.045961,7+Poole+Rd+London+E9+7AE
3,delivery_03,Eta James,"51 Whiston Road, London",E2 8RZ,51.534488,-0.066195,51+Whiston+Road+London+E2+8RZ
4,delivery_04,Chet Baker,"Adeyfield House, Cranwood St, Hoxton, London",EC1V 9PD,51.526541,-0.086816,Adeyfield+House+Cranwood+St+Hoxton+London+EC1V...
5,delivery_05,Louis Armstrong,"131 Highbury Quadrant Highbury East, London",N5 2TG,51.558769,-0.095979,131+Highbury+Quadrant+Highbury+East+London+N5+2TG
6,delivery_06,Aretha Franklin,"23 Fladbury Rd, London",N15 6SB,51.578274,-0.085052,23+Fladbury+Rd+London+N15+6SB
7,delivery_07,Akira Kosemura,"10 Ladbroke House 62-66 Highbury Grove, Highbu...",N5 2AG,51.553173,-0.097935,10+Ladbroke+House+62-66+Highbury+Grove+Highbur...
8,delivery_08,Ella Fitzgerald,"45 Lampard Grove, Cazenove, London",N16 6XB,51.567425,-0.070328,45+Lampard+Grove+Cazenove+London+N16+6XB
9,delivery_09,Nancy Wilson,"17 Harrowgate Rd, London",E9 7BS,51.543033,-0.038413,17+Harrowgate+Rd+London+E9+7BS


In [27]:
def create_distance_matrix(data):
    addresses = data["addresses"]

    # Distance Matrix API only accepts 100 elements per request, so get rows in multiple requests.
    max_elements = 100
    num_addresses = len(addresses) # 16 in this example.
    
    # Maximum number of rows that can be computed per request (6 in this example).
    max_rows = max_elements // num_addresses
    
    # num_addresses = q * max_rows + r (q = 2 and r = 4 in this example).
    q, r = divmod(num_addresses, max_rows)
    dest_addresses = addresses
    distance_matrix = []
    
    # Send q requests, returning max_rows rows per request.
    for i in range(q):
        origin_addresses = addresses[i * max_rows: (i + 1) * max_rows]
        response = send_request(origin_addresses, dest_addresses, API_KEY)
        distance_matrix += build_distance_matrix(response)

    # Get the remaining remaining r rows, if necessary.
    if r > 0:
        origin_addresses = addresses[q * max_rows: q * max_rows + r]
        response = send_request(origin_addresses, dest_addresses, API_KEY)
        distance_matrix += build_distance_matrix(response)
        return distance_matrix

In [28]:
def send_request(origin_addresses, dest_addresses, API_key):
    """ Build and send request for the given origin and destination addresses."""
    
    def build_address_str(addresses):
        # Build a pipe-separated string of addresses
        address_str = ''
        
        for i in range(len(addresses) - 1):
            address_str += addresses[i] + '|'
        address_str += addresses[-1]
        return address_str

    request = 'https://maps.googleapis.com/maps/api/distancematrix/json?units=imperial'
    origin_address_str = build_address_str(origin_addresses)
    dest_address_str = build_address_str(dest_addresses)
    
    request = request + '&origins=' + origin_address_str + '&destinations=' + \
                       dest_address_str + '&key=' + API_KEY
    
    jsonResult = urllib.request.urlopen(request).read()
    response = json.loads(jsonResult)
    
    return response

In [29]:
def build_distance_matrix(response):
    distance_matrix = []

    for row in response['rows']:
        row_list = [row['elements'][j]['distance']['value'] for j in range(len(row['elements']))]
        distance_matrix.append(row_list)
    
    return distance_matrix

In [30]:
def create_data():
    """Creates the data."""
    data = {}
    data['addresses'] = list(basic_locations_df['api_address'])
    return data

In [31]:
data = create_data()

In [32]:
addresses = data['addresses']
addresses

['Unit+8+Hamlet+Industrial+Estate+96+White+Post+Ln+London+E9+5EN',
 '94+Woolridge+Way+Hackney+London+E9+6PR',
 '7+Poole+Rd+London+E9+7AE',
 '51+Whiston+Road+London+E2+8RZ',
 'Adeyfield+House+Cranwood+St+Hoxton+London+EC1V+9PD',
 '131+Highbury+Quadrant+Highbury+East+London+N5+2TG',
 '23+Fladbury+Rd+London+N15+6SB',
 '10+Ladbroke+House+62-66+Highbury+Grove+Highbury+East+London+N5+2AG',
 '45+Lampard+Grove+Cazenove+London+N16+6XB',
 '17+Harrowgate+Rd+London+E9+7BS',
 '22+Turner+House+Corbyn+St+Finsbury+Park+London+N4+3DD']

In [33]:
distance_matrix = create_distance_matrix(data)
distance_matrix

[]

## VRP

In [19]:
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = distance_matrix
    data['num_vehicles'] = len(dispatch_crew_df)
    data['depot'] = 0 # index of collection point location
    return data

In [20]:
def print_solution(data, manager, routing, solution):
    """Prints solution on console.
    Also creates solution dictionary
    """
    
    solution_dict = {}
    
    max_route_distance = 0
    
    for vehicle_id in range(data['num_vehicles']):
        
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        
        while not routing.IsEnd(index):
            plan_output += ' {} -> '.format(manager.IndexToNode(index))
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        
        plan_output += '{}\n'.format(manager.IndexToNode(index))
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        print(plan_output)
        
        max_route_distance = max(route_distance, max_route_distance)
    
    print('Maximum of the route distances: {}m'.format(max_route_distance))

In [21]:
# Instantiate the data problem.
data = create_data_model()

In [22]:
# Create the routing index manager.
manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])
manager

<ortools.constraint_solver.pywrapcp.RoutingIndexManager; proxy of <Swig Object of type 'operations_research::RoutingIndexManager *' at 0x11e325180> >

In [23]:
# Create Routing Model.
routing = pywrapcp.RoutingModel(manager)
routing

<ortools.constraint_solver.pywrapcp.RoutingModel; proxy of <Swig Object of type 'operations_research::RoutingModel *' at 0x11e2fe0c0> >

In [24]:
# Create and register a transit callback.
def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    
    return data['distance_matrix'][from_node][to_node]

In [25]:
transit_callback_index = routing.RegisterTransitCallback(distance_callback)
transit_callback_index

1

In [26]:
# Define cost of each arc.
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)


In [27]:
# Add Distance constraint.
dimension_name = 'Distance'
routing.AddDimension(
    transit_callback_index,
    0,  # no slack
    3000000,  # vehicle maximum travel distance
    True,  # start cumul to zero
    dimension_name)
distance_dimension = routing.GetDimensionOrDie(dimension_name)
distance_dimension.SetGlobalSpanCostCoefficient(100)

In [28]:
# Setting first solution heuristic.
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

In [29]:
# Solve the problem.
solution = routing.SolveWithParameters(search_parameters)
solution

<ortools.constraint_solver.pywrapcp.Assignment; proxy of <Swig Object of type 'operations_research::Assignment *' at 0x11e2fef60> >

In [30]:
# Print solution on console.
if solution:
    print_solution(data, manager, routing, solution)

Route for vehicle 0:
 0 ->  6 ->  4 -> 0
Distance of the route: 13321m

Route for vehicle 1:
 0 ->  9 ->  10 ->  1 -> 0
Distance of the route: 11042m

Route for vehicle 2:
 0 ->  5 -> 0
Distance of the route: 13037m

Route for vehicle 3:
 0 ->  3 -> 0
Distance of the route: 13209m

Route for vehicle 4:
 0 ->  2 ->  7 ->  8 -> 0
Distance of the route: 9525m

Maximum of the route distances: 13321m


In [31]:
def get_routes(solution, routing, manager):
    """Get vehicle routes from a solution and store them in an array."""
    # Get vehicle routes and store them in a two dimensional array whose
    # i,j entry is the jth location visited by vehicle i along its route.
    routes = []
    for route_nbr in range(routing.vehicles()):
        index = routing.Start(route_nbr)
        route = [manager.IndexToNode(index)]
        while not routing.IsEnd(index):
            index = solution.Value(routing.NextVar(index))
            route.append(manager.IndexToNode(index))
        routes.append(route)
    return routes

In [32]:
routes = get_routes(solution, routing, manager)

# Display the routes.
for i, route in enumerate(routes):
      print('Route', i, route)

Route 0 [0, 6, 4, 0]
Route 1 [0, 9, 10, 1, 0]
Route 2 [0, 5, 0]
Route 3 [0, 3, 0]
Route 4 [0, 2, 7, 8, 0]


In [33]:
routes

[[0, 6, 4, 0], [0, 9, 10, 1, 0], [0, 5, 0], [0, 3, 0], [0, 2, 7, 8, 0]]

In [34]:
routes_without_collection_point = [route[1:-1] for route in routes]
routes_without_collection_point

[[6, 4], [9, 10, 1], [5], [3], [2, 7, 8]]

In [35]:
# Drop empty routes

routes_without_collection_point = [route for route in routes_without_collection_point if route]
routes_without_collection_point

[[6, 4], [9, 10, 1], [5], [3], [2, 7, 8]]

In [36]:
def get_route_number_of_collection_point(current_route_number):
    for route_num, route in enumerate(routes_without_collection_point):
        if current_route_number in route:
            return int(route_num), route.index(current_route_number)

In [37]:
current_route_number = 5
get_route_number_of_collection_point(current_route_number)

(2, 0)

In [38]:
# Add route number to origin df

# Create column from index number (index zero is actually the collection point,
# so delivery points have numbers matching id_num)
basic_locations_df = basic_locations_df.reset_index().rename(columns={'index': 'route_num_order'})

# Use index number (delivery number) to find which route it's in
basic_locations_df['route_num_order'] = basic_locations_df['route_num_order'].apply(lambda x: get_route_number_of_collection_point(x))

# Drop the collection point row
basic_locations_df = basic_locations_df[~basic_locations_df['id_num'].str.contains('collect')]

basic_locations_df

,route_num_order,id_num,name,address,postcode,latitude,longitude,api_address
1,"(1, 2)",delivery_01,Louise Kirkham,25 Coniston Walk,E9 6EP,51.5511,-0.048096,25+Coniston+Walk+E9+6EP
2,"(4, 0)",delivery_02,Idris Mohammed,"23 Speldhurst Rd, Hackney, London",E9 7EH,51.5391,-0.046886,23+Speldhurst+Rd+Hackney+London++E9+7EH
3,"(3, 0)",delivery_03,Syd Sampha,"Flat 20, Kensworth House, Cranwood St, Hoxton,...",EC1V 9PA,51.5265,-0.086916,Flat+20+Kensworth+House+Cranwood+St+Hoxton+Lon...
4,"(0, 1)",delivery_04,Dwight Sykes,"35 Oldfield Rd, Stoke Newington, London",N16 0RR,51.5587,-0.079139,35+Oldfield+Rd+Stoke+Newington+London+N16+0RR
5,"(2, 0)",delivery_05,Gene Williams,"28 Wellington Ave, London",N15 6AS,51.5779,-0.067773,28+Wellington+Ave+London++N15+6AS
6,"(0, 0)",delivery_06,Mavis John,"126 Shakspeare Walk, Stoke Newington, London",N16 8TA,51.5564,-0.081687,126+Shakspeare+Walk+Stoke+Newington+London++N1...
7,"(4, 1)",delivery_07,Yazmin Lacey,"71 Mapledene Rd, Hackney, London",E8 3JW,51.5418,-0.068707,71+Mapledene+Rd+Hackney+London++E8+3JW
8,"(4, 2)",delivery_08,Alice Smith,"49 Darnley Rd, Hackney, London",E9 6QH,51.544,-0.051267,49+Darnley+Rd+Hackney+London++E9+6QH
9,"(1, 0)",delivery_09,Mr Ronson,"42 Monteagle Way, Lower Clapton, London",E5 8PH,51.558,-0.062855,42+Monteagle+Way+Lower+Clapton+London++E5+8PH
10,"(1, 1)",delivery_10,Mrs Collins,"49 Reighton Rd, Clapton, London",E5 8SQ,51.5615,-0.06244,49+Reighton+Rd+Clapton+London++E5+8SQ


In [39]:
# Split out route number and order into two columns

basic_locations_df['route_num'] = basic_locations_df['route_num_order'].apply(lambda x: x[0])
basic_locations_df['order'] = basic_locations_df['route_num_order'].apply(lambda x: x[1])

# Sort values by route number and order
basic_locations_df = basic_locations_df.sort_values(by=['route_num', 'order'])

basic_locations_df

,route_num_order,id_num,name,address,postcode,latitude,longitude,api_address,route_num,order
6,"(0, 0)",delivery_06,Mavis John,"126 Shakspeare Walk, Stoke Newington, London",N16 8TA,51.5564,-0.081687,126+Shakspeare+Walk+Stoke+Newington+London++N1...,0,0
4,"(0, 1)",delivery_04,Dwight Sykes,"35 Oldfield Rd, Stoke Newington, London",N16 0RR,51.5587,-0.079139,35+Oldfield+Rd+Stoke+Newington+London+N16+0RR,0,1
9,"(1, 0)",delivery_09,Mr Ronson,"42 Monteagle Way, Lower Clapton, London",E5 8PH,51.558,-0.062855,42+Monteagle+Way+Lower+Clapton+London++E5+8PH,1,0
10,"(1, 1)",delivery_10,Mrs Collins,"49 Reighton Rd, Clapton, London",E5 8SQ,51.5615,-0.06244,49+Reighton+Rd+Clapton+London++E5+8SQ,1,1
1,"(1, 2)",delivery_01,Louise Kirkham,25 Coniston Walk,E9 6EP,51.5511,-0.048096,25+Coniston+Walk+E9+6EP,1,2
5,"(2, 0)",delivery_05,Gene Williams,"28 Wellington Ave, London",N15 6AS,51.5779,-0.067773,28+Wellington+Ave+London++N15+6AS,2,0
3,"(3, 0)",delivery_03,Syd Sampha,"Flat 20, Kensworth House, Cranwood St, Hoxton,...",EC1V 9PA,51.5265,-0.086916,Flat+20+Kensworth+House+Cranwood+St+Hoxton+Lon...,3,0
2,"(4, 0)",delivery_02,Idris Mohammed,"23 Speldhurst Rd, Hackney, London",E9 7EH,51.5391,-0.046886,23+Speldhurst+Rd+Hackney+London++E9+7EH,4,0
7,"(4, 1)",delivery_07,Yazmin Lacey,"71 Mapledene Rd, Hackney, London",E8 3JW,51.5418,-0.068707,71+Mapledene+Rd+Hackney+London++E8+3JW,4,1
8,"(4, 2)",delivery_08,Alice Smith,"49 Darnley Rd, Hackney, London",E9 6QH,51.544,-0.051267,49+Darnley+Rd+Hackney+London++E9+6QH,4,2


## Plotting on Map

In [34]:
basic_locations_df = pd.read_csv('data/output/selected_locations_solution.csv')
basic_locations_df.head()

,route_num_order,id_num,name,address,postcode,latitude,longitude,api_address,route_num,order
0,"(0, 0)",delivery_08,Ella Fitzgerald,"45 Lampard Grove, Cazenove, London",N16 6XB,51.567425,-0.070328,45+Lampard+Grove+Cazenove+London+N16+6XB,0,0
1,"(0, 1)",delivery_07,Akira Kosemura,"10 Ladbroke House 62-66 Highbury Grove, Highbu...",N5 2AG,51.553173,-0.097935,10+Ladbroke+House+62-66+Highbury+Grove+Highbur...,0,1
2,"(0, 2)",delivery_04,Chet Baker,"Adeyfield House, Cranwood St, Hoxton, London",EC1V 9PD,51.526541,-0.086816,Adeyfield+House+Cranwood+St+Hoxton+London+EC1V...,0,2
3,"(0, 3)",delivery_03,Eta James,"51 Whiston Road, London",E2 8RZ,51.534488,-0.066195,51+Whiston+Road+London+E2+8RZ,0,3
4,"(0, 4)",delivery_01,Gia Marrgaret,"94 Woolridge Way, Hackney, London",E9 6PR,51.542348,-0.049885,94+Woolridge+Way+Hackney+London+E9+6PR,0,4


In [39]:
fig_2 = go.Figure()

# Map layout
fig_2.update_layout(
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        accesstoken=my_mapbox_access_token,
        style='light',
        center={'lon': np.mean(delivery_points_df['longitude']),
                'lat': np.mean(delivery_points_df['latitude'])},
        zoom=11
    ),
)

for route_num, route_df in basic_locations_df.groupby('route_num'):

    # Add collection point to group df
    route_df = pd.concat([collection_point_df, route_df]).reset_index(drop=True)
        
    # Add route to map
    fig_2.add_trace(go.Scattermapbox(
            lat=route_df['latitude'],
            lon=route_df['longitude'],
            mode='lines+markers',
            marker=go.scattermapbox.Marker(
                size=13,
            ),
            hoverinfo='text',
            text='Stop ' + route_df.index.astype(str) + ' - ' + route_df['address'],

            name='Route ' + str(route_num)
        ))
    

# fig_2.show()
fig_2

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hoverinfo': 'text',
              'lat': array(['51.542540', 51.567425, 51.553173, 51.526541, 51.534488, 51.542348,
                            51.543422, 51.543033], dtype=object),
              'lon': array(['-0.023000', -0.070328, -0.097935, -0.086816, -0.066195, -0.049885,
                            -0.045961, -0.038413], dtype=object),
              'marker': {'size': 13},
              'mode': 'lines+markers',
              'name': 'Route 0',
              'text': array(['Stop 0 - Unit 8 Hamlet Industrial Estate, 96 White Post Ln, London',
                             'Stop 1 - 45 Lampard Grove, Cazenove, London',
                             'Stop 2 - 10 Ladbroke House 62-66 Highbury Grove, Highbury East, London',
                             'Stop 3 - Adeyfield House, Cranwood St, Hoxton, London',
                             'Stop 4 - 51 Whiston Road, London',
                             'Stop 5 - 94 Woolridge Way, Hackney, London',
                             'Stop 6 - 7 Poole Rd, London', 'Stop 7 - 17 Harrowgate Rd, London'],
                            dtype=object),
              'type': 'scattermapbox'},
             {'hoverinfo': 'text',
              'lat': array(['51.542540', 51.558769, 51.568494, 51.578274], dtype=object),
              'lon': array(['-0.023000', -0.095979, -0.117942, -0.085052], dtype=object),
              'marker': {'size': 13},
              'mode': 'lines+markers',
              'name': 'Route 1',
              'text': array(['Stop 0 - Unit 8 Hamlet Industrial Estate, 96 White Post Ln, London',
                             'Stop 1 - 131 Highbury Quadrant Highbury East, London',
                             'Stop 2 - 22 Turner House, Corbyn St, Finsbury Park, London',
                             'Stop 3 - 23 Fladbury Rd, London'], dtype=object),
              'type': 'scattermapbox'}],
    'layout': {'autosize': True,
               'hovermode': 'closest',
               'mapbox': {'accesstoken': ('pk.eyJ1IjoibG91aXNla2lya2hhbSI' ... 'ZHVlIn0.NeE7I8fu5zglEoeFc_eirQ'),
                          'center': {'lat': 51.551596700000005, 'lon': -0.0754506},
                          'style': 'light',
                          'zoom': 11},
               'template': '...'}
})